<a href="https://colab.research.google.com/github/nilesh0109/CV2_SoSe_19/blob/master/Saliency_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import zipfile
from google.colab import files, drive

drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/CV2 exercies/Archive.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
import numpy as np
import os
import tensorflow as tf
import imageio
from __future__ import division
from skimage.transform import resize
from sklearn.model_selection import KFold

In [0]:
ROOT= '/tmp'
NUM_IMAGES = 1200
VAL_NUM_IMAGES = 400

def load_train_data():
  training_img_directory = ROOT+'/data/train/images'
  training_fixation_directory = ROOT+'/data/train/fixations'
  val_img_directory = ROOT+'/data/val/images'
  val_fixation_directory = ROOT+'/data/val/fixations'
  train_imgs = np.zeros((NUM_IMAGES + VAL_NUM_IMAGES, 180, 320, 3), dtype=np.uint8)
  train_fixations = np.zeros((NUM_IMAGES + VAL_NUM_IMAGES, 180, 320, 1), dtype=np.uint8)
  
  for i in range(1, NUM_IMAGES + 1):
    img_file = os.path.join(training_img_directory, '{:04d}.jpg'.format(i))
    fixation_file = os.path.join(training_fixation_directory, '{:04d}.jpg'.format(i))
    train_imgs[i-1] = imageio.imread(img_file)
    fixation = imageio.imread(fixation_file)
    train_fixations[i-1] = np.expand_dims(fixation, -1) # adds singleton dimension so fixation size is (180,320,1)
  
  for j in range(i + 1, i + VAL_NUM_IMAGES + 1):
    img_file = os.path.join(val_img_directory, '{:04d}.jpg'.format(j))
    fixation_file = os.path.join(val_fixation_directory, '{:04d}.jpg'.format(j))
    train_imgs[j-1] = imageio.imread(img_file)
    fixation = imageio.imread(fixation_file)
    train_fixations[j-1] = np.expand_dims(fixation, -1) # adds singleton dimension so fixation size is (180,320,1)
    
  return train_imgs, train_fixations

# Generator function will output one (image, target) tuple at a time,
# and shuffle the data for each new epoch
def data_generator(imgs, targets):
	while True: # produce new epochs forever
		# Shuffle the data for this epoch
		idx = np.arange(imgs.shape[0])
		np.random.shuffle(idx)

		imgs = imgs[idx]
		targets = targets[idx]
		for i in range(imgs.shape[0]):
			yield imgs[i], targets[i]

def get_batch_from_generator(gen, batchsize):
	batch_imgs = []
	batch_fixations = []
	for i in range(batchsize):
		img, target = gen.__next__()
		batch_imgs.append(img)
		batch_fixations.append(target)
	return np.array(batch_imgs), np.array(batch_fixations)  


In [0]:
#tf.reset_default_graph()

'''
Model class
'''

class saliencyModel:
  def __init__(self, model_weights=None, learning_rate=1e-4, batch_size=32, num_epochs=100, prior_downsampling_factor=10):
    self.dir_path = 'drive/My Drive/Colab Notebooks/CV2 exercies/'
    self.model_path = self.dir_path+'model_cptk/my-model1'
    self.lr_rate = learning_rate
    self.batch_size = batch_size
    self.num_epochs = num_epochs
    self.prior_downsampling_factor = prior_downsampling_factor
    self.input_h = 180
    self.input_w = 320
    self.prior_h = self.input_h /(2 * prior_downsampling_factor)
    self.prior_w = self.input_w / (2 * prior_downsampling_factor)
    self.reg_lambda = 1 / (self.prior_h * self.prior_w)
    self.input_images_placeholder = tf.placeholder(tf.uint8, [None, self.input_h, self.input_w, 3])
    self.target_images_placeholder = tf.placeholder(tf.uint8, [None, self.input_h, self.input_w, 1])
    
    if model_weights is not None:
      self.load_weights(model_weights)
    
  def load_weights(self, model_weights):
    vgg_weight_file = model_weights
    self.weights = np.load(vgg_weight_file)
  
  def setup(self, mode= 'Train'):
    
    with tf.name_scope('preprocessing') as scope:
      input_imgs = tf.image.convert_image_dtype(self.input_images_placeholder, tf.float32) * 255
      fixations_normalized = tf.image.convert_image_dtype(self.target_images_placeholder, tf.float32)
      mean = tf.constant([123.68 , 116.779 , 103.939], dtype = tf.float32, shape =[1,1,1,3], name ='img_mean')
      imgs_normalized = input_imgs - mean

    with tf.name_scope('conv1_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv1_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv1_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(imgs_normalized, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv1_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv1_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv1_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('pool1') as scope:
      pool = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(2,2), padding='same')

    with tf.name_scope('conv2_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv2_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv2_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv2_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv2_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv2_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('pool2') as scope:
      pool2 = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(2,2), padding='same')

    with tf.name_scope('conv3_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv3_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv3_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv3_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv3_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv3_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('conv3_3') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv3_3_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv3_3_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)

    with tf.name_scope('pool3') as scope:
      pool3 = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(1,1), padding='same')

    with tf.name_scope('conv4_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv4_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv4_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool3, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      
    with tf.name_scope('conv4_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv4_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv4_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
    
    with tf.name_scope('conv4_3') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv4_3_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv4_3_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      
    with tf.name_scope('pool4') as scope:
      pool4 = tf.layers.max_pooling2d(act, pool_size=(2,2), strides=(1,1), padding='same')
    
    with tf.name_scope('conv5_1') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv5_1_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv5_1_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(pool4, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      
    with tf.name_scope('conv5_2') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv5_2_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv5_2_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
    
    with tf.name_scope('conv5_3') as scope:
      kernel = tf.Variable(initial_value=self.weights['conv5_3_W'], trainable=False, name="weights")
      biases = tf.Variable(initial_value=self.weights['conv5_3_b'], trainable=False, name="biases")
      conv = tf.nn.conv2d(act, kernel, [1, 1, 1, 1], padding='SAME')
      out = tf.nn.bias_add(conv, biases)
      act = tf.nn.relu(out, name=scope)
      conv5_3 = act
      
    with tf.name_scope('concat_featuremaps') as scope:
      concatenated_feature_maps = tf.concat([pool2, pool3, pool4, conv5_3], axis=3)
      print('IN_TRAINING_MODE is',mode =='Train')
      regularized_feature_maps = tf.layers.dropout(concatenated_feature_maps, rate=0.5, training= mode =='Train')

    with tf.name_scope('featuremaps_conv1') as scope:
      my_regularizer = tf.contrib.layers.l2_regularizer(1e-5)
      act_featuremaps_conv1 = tf.layers.conv2d(regularized_feature_maps, filters=64, kernel_size=(3,3), padding='SAME', activation=tf.nn.relu, name='featuremaps_conv1', kernel_regularizer = my_regularizer, reuse=tf.AUTO_REUSE)

    with tf.name_scope('featuremaps_conv2') as scope:
      act_featuremaps_conv2 = tf.layers.conv2d(act_featuremaps_conv1, filters=1, kernel_size=(1,1), activation=tf.nn.relu, name='featuremaps_conv2', kernel_regularizer = my_regularizer, reuse=tf.AUTO_REUSE)

    with tf.name_scope('Learned_prior') as scope:
      prior_shape = (1, self.prior_h, self.prior_w,1)
      prior = tf.Variable(tf.ones(prior_shape), name="prior", trainable=True)
      upsampled_prior = tf.image.resize_bilinear(prior, size=act_featuremaps_conv2.shape[1:3])
      saliency_mixed = tf.multiply(act_featuremaps_conv2, upsampled_prior)
      saliency_raw = tf.nn.relu(saliency_mixed)
      
    with tf.name_scope('loss') as scope:
      upsampled_saliency = tf.image.resize_bilinear(saliency_raw, size=fixations_normalized.shape[1:3])
      # normalize saliency
      max_value_per_image = tf.reduce_max(upsampled_saliency, axis=[1,2,3], keepdims=True)
      predicted_saliency = (upsampled_saliency / max_value_per_image)
      
      # Loss function from Cornia et al. (2016) [with higher weight for salient pixels]
      alpha = 1.01
      weight = 1.0 / (alpha - fixations_normalized)
      loss = tf.losses.mean_squared_error(labels=fixations_normalized, 
                        predictions=predicted_saliency, 
                        weights=weight)
      regularizer = tf.nn.l2_loss(1 - prior)
      loss += tf.reduce_mean(self.reg_lambda * regularizer)
      l2_loss = tf.losses.get_regularization_loss() 
      loss += l2_loss
      
    return predicted_saliency, loss
    
  def train(self, t_imgs, t_fixations):
    
    pred_val ,loss_op_val = self.setup(mode='Test')
    pred ,loss_op = self.setup(mode='Train')
    training_loss = []
    val_loss = []
    
    # Optimizer settings from Cornia et al. (2016) [except for decay]
    optimizer = tf.train.MomentumOptimizer(learning_rate=self.lr_rate, momentum=0.4, use_nesterov=True)
    minimize_op = optimizer.minimize(loss_op)
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
      #writer = tf.summary.FileWriter(logdir="./", graph=sess.graph)
      sess.run(tf.global_variables_initializer())
      saver.restore(sess, "/model/latest1-900")
      for epoch_ind in range(0, self.num_epochs, 5):
        kf = KFold(n_splits=5)
        for train_index, val_index in kf.split(t_imgs):
          train_gen = data_generator(t_imgs[train_index], t_fixations[train_index])
          val_gen = data_generator(t_imgs[val_index], t_fixations[val_index])
          
          num_batches = len(train_index) // self.batch_size
          for b in range(num_batches):
            batch_imgs, batch_fixations = get_batch_from_generator(train_gen, self.batch_size)
            predication, batch_loss,_ = sess.run([pred, loss_op, minimize_op], feed_dict={self.input_images_placeholder: batch_imgs, self.target_images_placeholder: batch_fixations})
          training_loss.append(batch_loss)
          
        if epoch_ind % 5 == 0:
          #writer.add_summary(l_summary, global_step=b)
          print('epoch {0} done: TRAINING batch loss {1}'.format(epoch_ind, batch_loss))
            
        if epoch_ind % 100 == 0:
          val_batch_imgs, val_batch_fixations = get_batch_from_generator(val_gen, self.batch_size)
          prediction_val, batch_loss_val = sess.run([pred_val, loss_op_val], feed_dict={self.input_images_placeholder: val_batch_imgs, self.target_images_placeholder: val_batch_fixations})
          print('epoch {0} VALIDATION batch loss {1}'.format(epoch_ind, batch_loss_val))
          val_loss.append(batch_loss_val)
          
        if epoch_ind % 100 == 0 and epoch_ind != 0:
          save_path = saver.save(sess, 'model/latest1', global_step=epoch_ind)
      save_path = saver.save(sess, 'model/latest1')
      
      
  def test(self, test_imgs):
    pred_saliency, l = self.setup(mode='Test')
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
      #writer = tf.summary.FileWriter(logdir="./", graph=sess.graph)
      saver.restore(sess, 'model/latest1')
      saliency = sess.run(pred_saliency, feed_dict={self.input_images_placeholder: test_imgs})
      for i in range(len(test_imgs)):
        print('saving images')
        saliency_img = sess.run(tf.image.convert_image_dtype(saliency[i], tf.uint8))
        imageio.imwrite(str(i)+'.jpg', saliency_img)
    
    

In [0]:
tf.reset_default_graph()

train_imgs, train_fixations = load_train_data()

my_model = saliencyModel(model_weights='/tmp/Model/VGG16/vgg16-conv-weights.npz', learning_rate=1e-9, num_epochs=601, batch_size=32)

my_model.train(train_imgs, train_fixations)
#my_model.train(train_imgs, train_fixations)

IN_TRAINING_MODE is False
IN_TRAINING_MODE is True
epoch 0 done: TRAINING batch loss 0.020769167691469193
epoch 0 VALIDATION batch loss 0.01909538358449936
epoch 5 done: TRAINING batch loss 0.01939050294458866
epoch 10 done: TRAINING batch loss 0.019853781908750534
epoch 15 done: TRAINING batch loss 0.021648284047842026
epoch 20 done: TRAINING batch loss 0.023059943690896034
epoch 25 done: TRAINING batch loss 0.021401621401309967
epoch 30 done: TRAINING batch loss 0.01946244202554226
epoch 35 done: TRAINING batch loss 0.01944793201982975
epoch 40 done: TRAINING batch loss 0.021813172847032547
epoch 45 done: TRAINING batch loss 0.021766619756817818
epoch 50 done: TRAINING batch loss 0.019501768052577972
epoch 55 done: TRAINING batch loss 0.020478906109929085
epoch 60 done: TRAINING batch loss 0.019124040380120277
epoch 65 done: TRAINING batch loss 0.02035260573029518
epoch 70 done: TRAINING batch loss 0.019317541271448135
epoch 75 done: TRAINING batch loss 0.01885630376636982
epoch 80 d

W0614 12:50:54.475792 139650139862912 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


epoch 600 VALIDATION batch loss 0.01582648605108261
epoch 605 done: TRAINING batch loss 0.017552973702549934
epoch 610 done: TRAINING batch loss 0.016805170103907585
epoch 615 done: TRAINING batch loss 0.018477067351341248
epoch 620 done: TRAINING batch loss 0.01615270972251892
epoch 625 done: TRAINING batch loss 0.019445078447461128
epoch 630 done: TRAINING batch loss 0.0173631701618433
epoch 635 done: TRAINING batch loss 0.016551600769162178
epoch 640 done: TRAINING batch loss 0.01913217268884182
epoch 645 done: TRAINING batch loss 0.018713420256972313
epoch 650 done: TRAINING batch loss 0.01713373512029648
epoch 655 done: TRAINING batch loss 0.01899234764277935
epoch 660 done: TRAINING batch loss 0.01686106249690056
epoch 665 done: TRAINING batch loss 0.019459355622529984
epoch 670 done: TRAINING batch loss 0.019476836547255516
epoch 675 done: TRAINING batch loss 0.016698164865374565
epoch 680 done: TRAINING batch loss 0.016630826517939568
epoch 685 done: TRAINING batch loss 0.01864

In [0]:
tf.reset_default_graph()

train_imgs, train_fixations = load_train_data()

my_model = saliencyModel(model_weights='/tmp/Model/VGG16/vgg16-conv-weights.npz', learning_rate=1e-1, num_batches=1001, batch_size=16)

my_model.test(train_imgs[0:16])
